In [41]:
from copy import deepcopy
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
from src.jss_lite.jss_lite import jss_lite
import numpy as np
import time
class tree_node:
    def __init__(self, action=None,parent=None):
        self.action=action
        self.children=[]
        self.state=None
        self.parent=parent
        self.observation=None
        self.visited=False
        # extra attributes for search:
        self.leaf_closed=False
        
class tree_construct:
    def __init__(self):
        # change if you want to load klagenfurt ...
        self.load_klagen=True
        # threshold for values that are as good as the threshold*max_policy
        self.threshold=0.7
        curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
        inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
        instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
        #self.env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
        
        from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
        import JSSEnv
        import gym
        if self.load_klagen:
            self.env=jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
        else:
            self.env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))

        # create root
        self.root=tree_node()
        # get initial state into root; todo: change for multi instances, create all childs for instances
        self.root.observation=self.env.reset()
        self.root.state=self.env.get_state()
        self.done=False
        self.reward=0
        self.trainer=self.load_trainer()
        # path management
        # open leafs should store the leafs with highest rating
        self.open_leafs=[]
        self.open_leafs.append(self.root)
        self.current_path=self.root
        self.sol_pathes=None
    def insert(self,action=None,parent=None):
        parent.children.append(tree_node(action=action,parent=self))
        self.env.set_state(parent.state)
        parent.children[-1].observation,reward,done,info=self.env.step(action)
        parent.children[-1].state=self.env.get_state()
        if done:
            #this goes for klagenfurt jssp
            if self.load_klagen:
                needed_time=self.env.env.env.last_time_step
                optimal_value=666
                reward=1-((needed_time-optimal_value)/optimal_value)
            self.__on_done(100*reward)
            


    def __on_done(self,reward):
        self.done=True
        if reward > self.reward:
            self.reward=reward
    def load_trainer(self):
        from ray.rllib.agents.ppo import PPOTrainer
        from ray import tune
        import gym
        from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
        from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
        from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
        from src.jss_lite.jss_lite import jss_lite
        from ray import tune

        if self.load_klagen:
            from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
            import JSSEnv
            #Configure the algorithm.
            def env_creator(env_config):
                import JSSEnv
                env = jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
                return env
        else:
            curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
            inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
            instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
            def env_creator(env_config):
                #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
                env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
                #env=jss_lite(instance_path=inst)
                #env=jssp_light_obs_wrapper_no_action_mask(jss_lite(instance_path=inst))
                return env

        tune.register_env('custom_jssp',env_creator)
        config = {
            "env": "custom_jssp",
            "disable_env_checking":True,
            "num_workers": 4,
            "framework": "tf",
            "model": {
                "fcnet_hiddens": [256, 256],
                "fcnet_activation": "relu",
            },
            "evaluation_num_workers": 0,
            "evaluation_config": {
                "render_env": False,
            },
            
        }
        trainer = PPOTrainer(config=config)
        
        if self.load_klagen:
            trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-60')
        else:
            trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/custom_ppo/checkpoint-60')
        return trainer

    def propose_actions(self,state,observation):
        policy = self.trainer.get_policy()
        action, _, info = policy.compute_single_action(observation)
        logits = info['action_dist_inputs']
        self.env.set_state(state=state)
        action_mask = self.env.get_legal_actions().astype(np.float32)
        action_mask[action_mask==0] = - np.inf
        probs=np.multiply(logits,action_mask)
        probs[probs==np.inf]=-np.inf
        best_action_id = probs.argmax()  # deterministic
        highest_prob=probs.max()
        #todo: check if it is good here
        out,=np.where(probs>self.threshold*highest_prob)
        return out, best_action_id    


    def solve(self,max_pathes=None,max_time=None):
        # here comes the solving method with abort conditions
        start_time=time.time()
        p_time=time.time()
        self.sol_pathes=0

        if max_pathes==None:
            max_pathes=5000 
        if max_time==None:
            max_time=60
        #print(self.open_leafs)
        while self.open_leafs:
            self.sol_pathes+=1
            # abort conditions:
            node=self.open_leafs.pop()
            if self.sol_pathes > max_pathes or time.time()-start_time > max_time or self.reward==100:
                return True
            while not self.done:
                loop=0
                
                #possible_actions=np.random.choice(len(node.observation['action_mask']), p_r, p=(node.observation['action_mask'] / node.observation['action_mask'].sum()))
                possible_actions, best_action = self.propose_actions(node.state,node.observation)
                for action in set(possible_actions):
                    if action!=best_action:
                        #print(f"action is: {action}")
                        self.insert(action=action,parent=node)
                #assure that best action is insertet last
                self.insert(action=best_action,parent=node)
                for child in node.children:
                    if child != node.children[-1]:
                        #todo: check here if in action mask
                        self.open_leafs.append(child)
                while True:
                    if node.children[-1].visited==False:
                        node.children[-1].visited=True
                        node=node.children[-1]
                        
                        break
                    loop+=1
                    if loop > len(node.children):
                        print("all childrens visited; return done equal true")
                        self.done=True
            self.done=False
            




In [42]:
import ray
ray.shutdown()
init=tree_construct()
print("solve")
init.solve()
print(init.reward)
print(init.sol_pathes)

2022-10-27 12:45:56,531	INFO trainable.py:159 -- Trainable.setup took 20.601 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


solve
100.0
6


AttributeError: 'tree_construct' object has no attribute 'start_time'

In [26]:

from ray.rllib.agents.ppo import PPOTrainer
from ray import tune
import ray
import gym
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
from src.jss_lite.jss_lite import jss_lite
from ray import tune
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
# Configure the algorithm.
def env_creator(env_config):
    #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    #env=jss_lite(instance_path=inst)
    #env=jssp_light_obs_wrapper_no_action_mask(jss_lite(instance_path=inst))
    return env
from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
import JSSEnv
# #Configure the algorithm.
# def env_creator(env_config):
#     import JSSEnv
#     env = jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
#     return env

ray.shutdown()
# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)


config = {
    "env": "custom_jssp",
    "disable_env_checking":True,
    "num_workers": 4,
    "framework": "tf",
    "model": {
        "fcnet_hiddens": [256, 256],
        "fcnet_activation": "relu",
    },
    "evaluation_num_workers": 0,
    "evaluation_config": {
        "render_env": False,
    },
    
}
trainer = PPOTrainer(config=config)
#trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/ppo/checkpoint-4')
#trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-50')

for _ in range(60):
    print(_)
    trainer.train()
    #trainer.save_checkpoint("training_checkpoints/klagen_ppo")
    trainer.save_checkpoint("training_checkpoints/custom_ppo")




2022-10-27 12:18:59,371	INFO trainable.py:159 -- Trainable.setup took 14.187 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [23]:
for _ in range(1,61):
    print('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-'+str(_))
    trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-'+str(_))
    import numpy as np
    #trainer.save_checkpoint("ppo")
    episode_reward = 0
    #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    #env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    #env=env_creator("s")
    env=jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
    done = False
    obs = env.reset()
    iterations=0


    while not done:
        policy = trainer.get_policy()
        action, _, info = policy.compute_single_action(obs)
        logits = info['action_dist_inputs']
        action_mask = env.get_legal_actions().astype(np.float32)
        action_mask[action_mask==0] = - np.inf
        probs=np.multiply(logits,action_mask)
        probs[probs==np.inf]=-np.inf
        best_action_id = probs.argmax()  # deterministic
        #best_action_id = logits.argmax()
        #best_action_id = trainer.compute_single_action(obs)
        obs, reward, done, info = env.step(best_action_id)
        episode_reward += reward
        iterations += 1

        #print(best_action_id)
    #env.env.env.render()
    #print(iterations)
    #print(reward)
    needed_time=env.env.env.last_time_step
    optimal_value=666
    reward=1-((needed_time-optimal_value)/optimal_value)
    print(reward)
    #print(env.env.env.last_time_step-666)

/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-1
0.5810810810810811
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-2
0.6621621621621622
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-3
0.8108108108108107
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-4
0.7582582582582582
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-5
0.7867867867867868
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-6
0.7867867867867868
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-7
0.6861861861861862
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-8
0.7867867867867868
/Users/felix/sciebo/masterarbeit/progra/model-ba

In [16]:

needed_time=env.env.env.last_time_step
optimal_value=655
reward=1-((needed_time-optimal_value)/optimal_value)
print(reward)
#print(datetime.datetime.fromtimestamp(env.env.env.start_timestamp+env.env.env.last_time_step))
#print((env.env.env.start_timestamp))
#print(datetime.datetime.fromtimestamp(env.env.env.start_timestamp))

#print(env.env.env.last_time_step)
#print(datetime.datetime.fromtimestamp(env.env.env.last_time_step))
#print(10*60*60-9*60*60+6*60-54*60+56-2)



0.6931297709923664


In [ ]:
env.env.env.render()

In [ ]:
env_2=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
x,=env_2.observation_space['obs'].shape
print(x)

In [ ]:
print(iterations)
env.render()

In [ ]:
print(info)
print(len(info['action_dist_inputs']))

In [ ]:
import numpy as np
import time
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst=curr_dir + '/resources/jsp_instances/standard/ft06.txt'
instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
# Configure the algorithm.
def env_creator(env_config):
    env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    #env=jss_lite(instance_path=inst)
    return env
# get random actions:
reward=0
while reward <0.95:
#for _ in range(100):
    action_list=[]
    env = env_creator("s")

    obs = env.reset()
    # env2 is copy for later going evaluation
    counter=0
    done = False
    t=time.time()
    while not done:
        legal_action=obs['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        action_list.append(action)
        #print(action)
        #print(obs['action_mask'][action])
        obs, reward, done, _ = env.step(action)
        counter+=1
    #print(f"instance: {env.env.instance} in time: {time.time()-t}s")
    
    #env.render()
    #env.close()
print(counter)
print(reward)
env.render()

In [ ]:
import numpy as np
x=np.array([2,2,2,2,2,2,2,2,2,2,2,2,4])
print(x[x>=0.8*2].argmax())
y,=(np.where(x>=2))

for i in y:
    print(i)